# LSTM을 이용한 텍스트 생성
 - 데이터 다운로드: 캐글 New York Times Comments
 - ArticlesApril2018.csv

In [1]:
import numpy as np 
import tensorflow as tf
seed=2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

- 뉴욕타임즈 기사 

In [9]:
import pandas as pd
# df = pd.read_csv('data/ArticlesApril2018.csv', encoding='latin1')
df = pd.read_csv('/content/ArticlesApril2018.csv', encoding='latin1')
df.head(3)

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleadersâ Settlement Offer...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"âI understand that they could meet with us, ...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,Whatâs it like to eat at the second incarnat...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...


- 데이터 전처리

In [10]:
df.columns

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')

In [11]:
# headline 열 데이터만 사용, Null 검사
df.headline.isnull().sum()

0

In [12]:
headlines = [text for text in df.headline.values]
len(headlines)

1324

In [13]:
headlines[:5]

['Former N.F.L. Cheerleadersâ\x80\x99 Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [ ]:
# 노이즈 데이터('Unknown') 제거
headlines = [text for text in headlines if text != 'Unknown']
len(headlines)

In [14]:
# 구둣점 제거, 소문자로 변환
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [16]:
def preprocessing(s):
    s = s.encode('utf8').decode('ascii', 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

In [17]:
headlines = [preprocessing(s) for s in headlines]
headlines[:3]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained']

In [18]:
# 단어 집합을 만들고, 그 크기를 확인
t = Tokenizer()
t.fit_on_texts(headlines)
vocab_size = len(t.word_index) + 1
vocab_size

3495

In [19]:
# 학습에 사용될 샘플 시퀀스
sequences = []
for line in headlines:
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
sequences[:11]

[[100, 270],
 [100, 270, 372],
 [100, 270, 372, 1116],
 [100, 270, 372, 1116, 583],
 [100, 270, 372, 1116, 583, 53],
 [100, 270, 372, 1116, 583, 53, 7],
 [100, 270, 372, 1116, 583, 53, 7, 2],
 [100, 270, 372, 1116, 583, 53, 7, 2, 373],
 [100, 270, 372, 1116, 583, 53, 7, 2, 373, 11],
 [100, 270, 372, 1116, 583, 53, 7, 2, 373, 11, 1117],
 [101, 3]]

In [20]:
max_len = max(len(s) for s in sequences)
max_len

24

In [21]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         100,  270],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  100,
         270,  372],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,  100,  270,
         372, 1116]], dtype=int32)

In [22]:
X = sequences[:,:-1]
y = sequences[:,-1]
Y = to_categorical(y)
X.shape, Y.shape

((7803, 23), (7803, 3495))

## 모델 정의/설정/학습
- Embedding
- LSTM

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [26]:
# del model

In [27]:
# 임베딩 벡터 10차원, LSTM 노드 크기 128
model = Sequential([ 
    Embedding(vocab_size, 10, input_length=max_len-1),  ## ??
    LSTM(128),
    Dense(vocab_size, activation='softmax')
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 23, 10)            34950     
                                                                 
 lstm_2 (LSTM)               (None, 128)               71168     
                                                                 
 dense_2 (Dense)             (None, 3495)              450855    
                                                                 
Total params: 556,973
Trainable params: 556,973
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
)

In [29]:
%time history = model.fit(X, Y, epochs=200, verbose=0)

KeyboardInterrupt: ignored

## 모델 검증

In [ ]:
from my_util import sentence_generation

In [ ]:
print(sentence_generation(model, t, max_len, 'i', 10))
print(sentence_generation(model, t, max_len, 'epa', 10))
print(sentence_generation(model, t, max_len, 'former', 10))